In [ ]:
from scripts.ml_classifier import infer_label_from_path
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

df = pd.read_csv("dataset.csv")
df['gt_label'] = df['file'].apply(infer_label_from_path)

texts = df['text'].fillna("").tolist()
y = df['gt_label'].tolist()
vectorizer = TfidfVectorizer(max_features=15000, ngram_range=(1,2))
X = vectorizer.fit_transform(texts)

rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X, y, cv=cv, scoring='f1_macro')
print("RandomForest 5-fold F1-macro:", scores.mean())

rf.fit(X, y)
df['rf_label'] = rf.predict(vectorizer.transform(df['text'].fillna("").tolist()))
df.to_csv("dataset_rf.csv", index=False)
